In [ ]:
!sudo apt update
!apt-get install openjdk-21-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,118 kB]
Fetched 2,351 kB in 4s (583 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be up

In [ ]:
import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession.builder.master("local[*]").getOrCreate()


spark

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm")\
    .load("drive/MyDrive/final_ds_enc_v5_format.txt")

data

DataFrame[label: double, features: vector]

In [ ]:
# Split the data into train and test
splits = data.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

train

DataFrame[label: double, features: vector]

In [ ]:
# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
from pyspark.ml.classification import LogisticRegression


# create the trainer and set its parameters
trainer = LogisticRegression(maxIter=50, regParam=0.001, elasticNetParam=0.05)

  # train the model
model = trainer.fit(train)

  # compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.8356379635449402


layers = [384, 576, 384, 192, 96, 2], accuracy = 0.7851731952088055 (percettrone)

\newline

Regressione logistica (iter = 10, reg = 0.01, elasticnet = 0.05) = 0.786422874927057

Regressione logistica (iter = 10, reg = 0.01, elasticnet = 0.05) = 0.8241624365482233 (dataset v4)

Regressione logistica (iter = 10, reg = 0.001, elasticnet = 0.05) = 0.8263282571912014  (dataset v4)


svm sklearn SVC(C=10, gamma=1, kernel="rbf"):

precision    recall  f1-score   support

           0       0.86      0.83      0.84      7100
           1       0.85      0.88      0.86      7949

    accuracy                           0.85     15049
   macro avg       0.85      0.85      0.85     15049
weighted avg       0.85      0.85      0.85     15049



In [ ]:
!pip install scikit-learn
import pandas as pd


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

ds = pd.read_parquet("drive/MyDrive/final_ds_enc_v4.parquet")

train, test = train_test_split(ds, test_size=0.2, random_state=1234)
train_data, train_labels, test_data, test_labels = train["text"].tolist(), train["label"].tolist(), test["text"].tolist(), test["label"].tolist()

model = SVC(C=10, gamma=1, kernel="rbf")


model.fit(train_data, train_labels)

predictions = model.predict(test_data)
print(classification_report(test_labels, predictions))


In [ ]:
!pip install transformers
!pip install datasets


from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset


dataset = load_dataset("parquet", data_files="final_ds_wtext_v5.parquet", split='train')
dataset = dataset.train_test_split(test_size=0.2)
checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(row):
    return tokenizer(row["text"], truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/64671 [00:00<?, ? examples/s]

Map:   0%|          | 0/16168 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names


['labels', 'input_ids', 'attention_mask']

In [ ]:
!pip install torch

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=16, collate_fn=data_collator
)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

20210


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
!pip install tqdm

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/20210 [00:00<?, ?it/s]

In [ ]:
!pip install evaluate


import evaluate

acc = evaluate.load("accuracy")
prec = evaluate.load("precision")
f1 = evaluate.load("f1")
rec = evaluate.load("recall")

model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    acc.add_batch(predictions=predictions, references=batch["labels"])
    prec.add_batch(predictions=predictions, references=batch["labels"])
    f1.add_batch(predictions=predictions, references=batch["labels"])
    rec.add_batch(predictions=predictions, references=batch["labels"])


log = {"Accuracy": acc.compute(), "Precision": prec.compute(), "f1-score": f1.compute(), "Recall": rec.compute()}
log

In [ ]:
model.save_pretrained("tapmodelv2_roBERTa", from_pt=True)